# Ezega Data Analysis

Author: [Meheret Samuel](https://github.com/senadev42)

<hr>

### The Pre-amble
A few months ago a friend of mine, [Kidus](https://github.com/kidesleo),  decided to scrape ezega.com, which is a sort of index for bussinesses in ethiopia. The result was a jsonl file with about 8k entries that I've been sitting on with the vague goal of looking into but not finding the time to.

Finally have time now, and while doing data analysis with clean datasets taken off the internet has been fun, it's not time to work with something dirty and real.


## Setting Up

First let's import the libraries we're going to be using.

In [1]:
import pandas as pd
import re

And then the file itself, setting lines=True to indicate that newlines are being used as a delimiter here because this is jsonl.

In [2]:
df = pd.read_json('ezega_data.jsonl', lines=True)

Now let's take a peek at our data set.

In [3]:
df.head()

,business_title,business_image,business_location,business_url,business_description,business_numbers,ezega_url,category,sub_category
0,Bagel Corner Bakery,https://businessguide.ezega.com/images/noimage...,Addis Ababa,None,None,"[0116513067, 0911455510, +251116352780, 011550...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Resources
1,AWASH BAR & RESTURANT,https://businessguide.ezega.com/images/noimage...,Mekelle,None,None,"[0344402815, 0344418248, 0344415657]",https://businessguide.ezega.com/Default.aspx?a...,Auto,Resources
2,BITMAS General Trading PLC,https://businessguide.ezega.com/images/noimage...,"Ras Desta Damtwe St. NTO Building, 3rd floor, ...",None,None,"[+251115501364, +251911516445+2510911133182, +...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Water Services
3,ROB VIDEO,https://businessguide.ezega.com/images/noimage...,Mekelle,None,None,[0344400112],https://businessguide.ezega.com/Default.aspx?a...,Auto,Water Services
4,Tesfa PLWHA Association,https://businessguide.ezega.com/images/noimage...,Addis Ababa,None,None,"[0467730206, 0116558830, 2511613426, 011554039...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Shipping Companies


As expected, it's a bit messy. 

At a glance, 
- `business_url` and `business_description` look like they have a lot of values missing, 
- the bussiness titles are inconsistently capitalized,
- some of the locations are cities while some of them are full addresses
- and the bussiness numbers are arrays that seem to be a mix of the Country Code +251 suffix and the simple, local 0 suffix
- the categories don't seem super relevant to the companies name so I suspect it wasn't registered properly.

And there's probably other stuff that isn't immediately apparent from this quick look through, but let's work on cleaning this up first.

## Clean Up

### 1. Missing Values

> business_url and business_description look like they have a lot of values missing

Let's start up by seeing exactly how much is missing, and if there's any other fields that are missing data as well.

In [4]:
print("Number of columns: ", df.shape[0])

df.isnull().sum()

Number of columns:  8656


business_title             0
business_image             0
business_location          0
business_url            6754
business_description    7358
business_numbers           0
ezega_url                  0
category                   0
sub_category               0
dtype: int64

Okay, that's actually a lot of missing urls and descriptions. Dropping the rows in question isn't an option when they make up over half of our dataset, so we can just replace them with placeholders for now.

In [5]:
df['business_url'] = df['business_url'].fillna('N/A')
df['business_description'] = df['business_description'].fillna('No description available')

Let's check again.

In [6]:
df.isnull().sum()

business_title          0
business_image          0
business_location       0
business_url            0
business_description    0
business_numbers        0
ezega_url               0
category                0
sub_category            0
dtype: int64

In [7]:
df.head()

,business_title,business_image,business_location,business_url,business_description,business_numbers,ezega_url,category,sub_category
0,Bagel Corner Bakery,https://businessguide.ezega.com/images/noimage...,Addis Ababa,N/A,No description available,"[0116513067, 0911455510, +251116352780, 011550...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Resources
1,AWASH BAR & RESTURANT,https://businessguide.ezega.com/images/noimage...,Mekelle,N/A,No description available,"[0344402815, 0344418248, 0344415657]",https://businessguide.ezega.com/Default.aspx?a...,Auto,Resources
2,BITMAS General Trading PLC,https://businessguide.ezega.com/images/noimage...,"Ras Desta Damtwe St. NTO Building, 3rd floor, ...",N/A,No description available,"[+251115501364, +251911516445+2510911133182, +...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Water Services
3,ROB VIDEO,https://businessguide.ezega.com/images/noimage...,Mekelle,N/A,No description available,[0344400112],https://businessguide.ezega.com/Default.aspx?a...,Auto,Water Services
4,Tesfa PLWHA Association,https://businessguide.ezega.com/images/noimage...,Addis Ababa,N/A,No description available,"[0467730206, 0116558830, 2511613426, 011554039...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Shipping Companies


Well that fixes that.

### 2. Cleaning the title

> the bussiness titles are inconsistently capitalized,

This would be somewhat of a non-issue if I didn't plan on doing some analysis on the names themselves later. And it frankly feels like a tricky issue to solve. Let's look at the column and see what we're working with first.

In [8]:
def is_fully_capitalized(title):
    '''
    For every bussiness title:
    1. Split it into words using space as a separator
    2. Count the number of words int he title
    3. Check if each word is capital, and count the number of fully capital words
    4. If that is equal to the total amount of words then the title is full capitalized, return true
    '''
    words = title.split()
    total_words = len(words)
    capitalized_words = sum(word.isupper() for word in words)
    return capitalized_words == total_words

fully_capitalized_titles = df[df['business_title'].apply(is_fully_capitalized)]

print("Number of titles where all the words are capitalized")
print(fully_capitalized_titles['business_title'].shape[0])

Number of titles where all the words are capitalized
3948


Okay that's nearly half the titles. Not ideal. We also need to work around entity identifiers like PLC (or P.L.C), which themselves are inconsistent and need to be extracted to a different columns besides, so let's do that first.



#### 2.1 Extracting Entity Abbreviations

First let's get a full list of all of these as well as their possible variations, and then use that. But first let's trim any spaces on either side.

In [13]:
# List of substrings to check
substrings = ['PLC', 'P.L.C.', 'P.L.C', 'plc', 'plc.', 'Plc', 
              'S.Co.', 'S.Co', 'S.co.', 'S.CO.', 'S.C', 'SC', 'S.C.',
              'LTD.', 'LTD', 'LLC', 'L.L.C', 'PLC.']

# Function to check if title contains any substring
def contains_pattern(title):
    for substring in substrings:
        if substring in title:
            return True
    return False

# Filter DataFrame based on the substrings
titles_with_pattern = df[df['business_title'].apply(contains_pattern)]

print("Count: ", titles_with_pattern['business_title'].shape)

titles_with_pattern['business_title']

Count:  (12,)


1352                            ETHIO OTT WAREN PLC
1532                                       SYOM PLC
1660                            ETHIO OTT WAREN PLC
4316                       KADISCO GENERAL HOSPITAL
5885                           AMEGONIAN HOOL P.L.C
5940                           AMEGONIAN HOOL P.L.C
6074             SAFETY DRIVERS TRAINING HOOL S.CO.
6855                               B-AN TRADING PLC
6926                               B-AN TRADING PLC
6971                                         IT Plc
6992      FORTHRITT ELECTRO MECHANICAL SERVICES PLC
8435    ETHIOPIAN VENTURES / APPEX BOTTLING CO. PLC
Name: business_title, dtype: object

Let's work on extracting them to their own column, titled `business_entity_type`. 

This probably didn't catch everything but we'll refine it later, for now let's just move it out.

In [10]:
#df[['business_title', 'entity_identifier' ]].iloc[5885]

In [11]:
for index, row in df.iterrows():
    for substring in substrings:
        if substring in row['business_title']:
            df.at[index, 'entity_identifier'] = substring
            df.at[index, 'business_title'] = row['business_title'].replace(substring, '').replace(substring, '').strip()
        if substring in row['business_title']:
            df.at[index, 'entity_identifier'] = substring
            df.at[index, 'business_title'] = row['business_title'].replace(substring, '').replace(substring, '').strip()

df[['business_title', 'entity_identifier' ]]

,business_title,entity_identifier
0,Bagel Corner Bakery,NaN
1,AWASH BAR & RESTURANT,NaN
2,BITMAS General Trading,PLC
3,ROB VIDEO,NaN
4,Tesfa PLWHA Association,NaN
...,...,...
8651,ADA DAIRY COOPERATIVE SOCIETY,NaN
8652,Addis International Catering,Plc
8653,SOURIISH MARBLES,PLC
8654,ADAMA DEVELOPMENT,P.L.C


In [12]:
df[['business_title', 'entity_identifier']].notnull().sum()

business_title       8656
entity_identifier    2210
dtype: int64

Okay, I'd say we got about a fourth of the dataset with that operation. Let's crawl a bit through the others and see if there's any other identifiers we can pick up. 

It's not feasible to go title by title, so we'll have to look for shortcuts like say look for substrings that are smaller 3 letters or sub-strings that have periods in them.